In [122]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
import networkx as nx
import matplotlib.colors as mcolors
# warnings.simplefilter(action='ignore', category=FutureWarning)
# pd.set_option('display.max_rows', None)  # To display all rows
# pd.set_option('display.max_columns', None)  # To display all columns
import glob as glob
import math
from datetime import datetime 

I. Creating a dataframe with demographic information (age, education, gender, race)

* Basically just need to get the demographic csvs, then only include ones that have the RIDs from my dataset sheets, and then  

II. A4 demo sheet 
* A4_PTDEMOG_PRV2_07Mar2024 has age, racial, ethnic, education, marriage, 
* A4_SUBJINFO_PRV2_07Mar2024_age_apoe


In [123]:
a4_demo = pd.read_csv('../../data_paths_and_cleaning/data/demographic_csvs/A4/A4_PTDEMOG_PRV2_07Mar2024.csv')

In [124]:
a4_pet_suvrs = pd.read_csv('../../data_paths_and_cleaning/data/intermediate_data/a4/merged_a4_at_amy_pos_bi_harm.csv')

In [125]:
#need to change BID in a4_demo to be RID to be consistent 
a4_demo.rename(columns={'BID':'RID'}, inplace=True)

In [126]:
print(a4_demo)

            RID  VISCODE  EXAMDAY  PTGENDER  PTAGE  PTETHNIC PTRACE  PTLANG  \
0     B10018169        1      0.0         1  69.93         2      5       1   
1     B10021023        1      0.0         2  68.26         2      5       1   
2     B10048838        1      0.0         1  78.96         2      5       1   
3     B10081264        1      0.0         2  67.79         2      5       1   
4     B10082310        1      0.0         1  82.68         2      5       1   
...         ...      ...      ...       ...    ...       ...    ...     ...   
6940  B99957366        1      0.0         1  83.13         2      5       1   
6941  B99968206        1      0.0         1  70.45         2      5       1   
6942  B99971727        1      0.0         1  65.63         2      5       1   
6943  B99980935        1      0.0         2  73.84         2      5       1   
6944  B99987993        1      0.0         2  73.28         2      5       1   

      PTPLANG  PTEDUCAT  PTMARRY  PTNOTRT  PTHOME  

In [127]:
#a4_pet_suvrs has RID, use these RIDS to filter the a4_demo df to only the rows we need 
a4_demo_filtered = a4_demo.merge(a4_pet_suvrs[['RID']],on='RID',how='inner')

In [128]:
print(a4_demo_filtered) #looks good 

           RID  VISCODE  EXAMDAY  PTGENDER  PTAGE  PTETHNIC PTRACE  PTLANG  \
0    B10423472        1      0.0         2  78.25         2      5       1   
1    B10693248        1      0.0         2  65.94         2      5       1   
2    B10709603        1      0.0         1  74.15         2      5       1   
3    B10795731        1      0.0         1  69.23         2      5       1   
4    B10904689        1      0.0         2  68.21         2      5       1   
..         ...      ...      ...       ...    ...       ...    ...     ...   
368  B99525373        1      0.0         2  67.00         2      5       1   
369  B99649691        1      0.0         2  65.31         2      5       1   
370  B99696480        1      0.0         2  74.41         2      5       1   
371  B99860801        1      0.0         1  71.56         2      2       3   
372  B99870985        1      0.0         1  71.32         2      5       1   

     PTPLANG  PTEDUCAT  PTMARRY  PTNOTRT  PTHOME           upda

In [129]:
#pull out age, education, gender, race 
keep_cols = ['RID','PTGENDER','PTAGE','PTETHNIC','PTEDUCAT','PTRACE']
a4_demo_filtered=a4_demo_filtered[keep_cols]

In [130]:
#first match IDS from tau suvr data sheet 

In [131]:
print(a4_demo_filtered)

           RID  PTGENDER  PTAGE  PTETHNIC  PTEDUCAT PTRACE
0    B10423472         2  78.25         2      19.0      5
1    B10693248         2  65.94         2      16.0      5
2    B10709603         1  74.15         2      18.0      5
3    B10795731         1  69.23         2      18.0      5
4    B10904689         2  68.21         2      13.0      5
..         ...       ...    ...       ...       ...    ...
368  B99525373         2  67.00         2      20.0      5
369  B99649691         2  65.31         2      16.0      5
370  B99696480         2  74.41         2      12.0      5
371  B99860801         1  71.56         2      18.0      2
372  B99870985         1  71.32         2      14.0      5

[373 rows x 6 columns]


In [155]:
#check that there are no nans in the dataframe 
# Check if there are any NaN values in the entire DataFrame
has_nans = a4_demo_filtered.isna().any().any()

if has_nans:
    print("There are NaN values in the DataFrame.")
else:
    print("There are no NaN values in the DataFrame.")


There are no NaN values in the DataFrame.


III. ADNI demo sheet 
* PTDEMOG_07Mar2024 has no age but has visit date and date of birth so can use this to calc age, also has gender, education and race [PTRACCAT]
* can't immediately find apoe status so wait on that 
* this also has ADNI trial phase so in the manuscript it might be helpful to know list out which adni pahses these all came from

In [132]:
adni_demo = pd.read_csv('../../data_paths_and_cleaning/data/demographic_csvs/ADNI/PTDEMOG_07Mar2024.csv')

In [133]:
adni_pet_suvrs = pd.read_csv("../../data_paths_and_cleaning/data/intermediate_data/adni/merged_adni_at_amy_pos_bi_harm.csv")

In [134]:
print(adni_demo.columns)

Index(['PHASE', 'PTID', 'RID', 'VISCODE', 'VISCODE2', 'VISDATE', 'PTSOURCE',
       'PTGENDER', 'PTDOB', 'PTDOBYY', 'PTHAND', 'PTMARRY', 'PTEDUCAT',
       'PTWORKHS', 'PTWORK', 'PTNOTRT', 'PTRTYR', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTADBEG', 'PTCOGBEG', 'PTADDX', 'PTETHCAT', 'PTRACCAT',
       'PTIDENT', 'PTENGSPK', 'PTNLANG', 'PTENGSPKAGE', 'PTCLANG', 'PTLANGSP',
       'PTLANGWR', 'PTSPTIM', 'PTSPOTTIM', 'PTLANGPR1', 'PTLANGSP1',
       'PTLANGRD1', 'PTLANGWR1', 'PTLANGUN1', 'PTLANGPR2', 'PTLANGSP2',
       'PTLANGRD2', 'PTLANGWR2', 'PTLANGUN2', 'PTLANGPR3', 'PTLANGSP3',
       'PTLANGRD3', 'PTLANGWR3', 'PTLANGUN3', 'PTLANGPR4', 'PTLANGSP4',
       'PTLANGRD4', 'PTLANGWR4', 'PTLANGUN4', 'PTLANGPR5', 'PTLANGSP5',
       'PTLANGRD5', 'PTLANGWR5', 'PTLANGUN5', 'PTLANGPR6', 'PTLANGSP6',
       'PTLANGRD6', 'PTLANGWR6', 'PTLANGUN6', 'PTLANGTTL', 'PTETHCATH',
       'PTBORN', 'PTBIRPL', 'PTIMMAGE', 'PTIMMWHY', 'PTBIRPR', 'PTBIRGR', 'ID',
       'SITEID', 'USERDATE', 'USERDATE2', 'DD

In [135]:
#a4_pet_suvrs has RID, use these RIDS to filter the a4_demo df to only the rows we need 
adni_demo_filtered = adni_demo.merge(adni_pet_suvrs[['RID']],on='RID',how='inner')

In [136]:
print(adni_demo_filtered.iloc[98])

PHASE                                   ADNI2
PTID                               130_S_2403
RID                                      2403
VISCODE                                   v06
VISCODE2                                  m12
                                ...          
USERDATE2                                 NaN
DD_CRF_VERSION_LABEL                      NaN
LANGUAGE_CODE                             NaN
HAS_QC_ERROR                              NaN
update_stamp            2013-05-30 10:05:05.0
Name: 98, Length: 80, dtype: object


* there's not age, just visit date and dob, so need to calculate age using datetime 

In [137]:
adni_demo_filtered['VISDATE'] = pd.to_datetime(adni_demo_filtered['VISDATE'])

In [139]:
#check if birth year has nans 

In [140]:
print(adni_demo_filtered)

      PHASE        PTID   RID VISCODE VISCODE2    VISDATE  PTSOURCE  PTGENDER  \
0     ADNI1  023_S_0031    31      sc       sc 2005-10-03       1.0       2.0   
1    ADNIGO  023_S_0031    31      sc       sc 2010-12-01       1.0       2.0   
2    ADNIGO  023_S_0031    31     m60      m60 2010-12-01       1.0       NaN   
3     ADNI2  023_S_0031    31     v06      m72 2011-10-20       1.0       2.0   
4     ADNI1  127_S_0112   112      sc       sc 2006-01-04       1.0       1.0   
..      ...         ...   ...     ...      ...        ...       ...       ...   
476   ADNI3  033_S_7079  7079      sc       sc 2022-05-23       1.0       2.0   
477   ADNI3  033_S_7088  7088      sc       sc 2022-06-17       1.0       1.0   
478   ADNI3  033_S_7100  7100      sc       sc 2022-08-08       1.0       1.0   
479   ADNI4  033_S_7100  7100  4_init      NaN 2023-09-11       1.0       1.0   
480   ADNI3  035_S_7105  7105      sc       sc 2022-08-22       1.0       1.0   

       PTDOB  PTDOBYY  ... 

In [141]:
#check for nans that might be causing errors 

In [142]:
import pandas as pd
from datetime import datetime
#double check this 


# correcting the 'PTDOB' processing
# ensure that the 'PTDOB' column contains strings, and handle NaN values by skipping them
adni_demo_filtered['PTDOB'] = adni_demo_filtered['PTDOB'].apply(lambda x: datetime.strptime(x, "%m/%Y") if pd.notnull(x) and isinstance(x, str) else x)

# # replace NaN values with a NaT (Not a Time) in 'PTDOB' to maintain datetime type consistency
# adni_demo_filtered['PTDOB'] = pd.to_datetime(adni_demo_filtered['PTDOB'], errors='coerce')

# # for those rows where 'PTDOB' is not NaT, set the day to 1 (check if this is neccesary)
# adni_demo_filtered.loc[adni_demo_filtered['PTDOB'].notnull(), 'PTDOB'] = adni_demo_filtered['PTDOB'][adni_demo_filtered['PTDOB'].notnull()].apply(lambda x: x.replace(day=1))

# # now you can calculate the age, ensuring that 'PTDOB' is not NaT
# adni_demo_filtered['AGE'] = adni_demo_filtered.apply(
#     lambda row: (row['VISDATE'].year - row['PTDOB'].year - ((row['VISDATE'].month, row['VISDATE'].day) < (row['PTDOB'].month, row['PTDOB'].day))) if pd.notnull(row['PTDOB']) else None, axis=1
# )
#account for fractional ages 
# Now calculate the age more precisely with months as fractions of the year
adni_demo_filtered['AGE'] = adni_demo_filtered.apply(
    lambda row: (
        row['VISDATE'].year - row['PTDOB'].year +
        ((row['VISDATE'].month - row['PTDOB'].month) / 12) - 
        ((row['VISDATE'].month == row['PTDOB'].month) and (row['VISDATE'].day < row['PTDOB'].day) / 12)
    ) if pd.notnull(row['PTDOB']) else None, axis=1
)


In [143]:
print(adni_demo_filtered['AGE'].unique)

<bound method Series.unique of 0      77.750000
1      82.916667
2            NaN
3      83.750000
4      70.666667
         ...    
476    86.750000
477    70.000000
478    72.916667
479    74.000000
480    77.500000
Name: AGE, Length: 481, dtype: float64>


In [144]:
#pull out age, education, gender, race 
keep_cols_adni = ['RID','PTGENDER','AGE','PTETHCAT','PTEDUCAT','PTRACCAT']
adni_demo_filtered=adni_demo_filtered[keep_cols_adni]

* Now only keep the latest visit row to make sure that each patient only has one row in the df 

In [145]:
#function to filter the latest visit of a df 
def filter_latest_visit(df):
    last_visit_df = df.groupby('RID').tail(1) #get the last occurance of that RID value 
    last_visit_df = last_visit_df.reset_index(drop=True)
    return last_visit_df

In [146]:
adni_demo_filtered = filter_latest_visit(adni_demo_filtered)

In [151]:
print(adni_demo_filtered.iloc[1])

RID               112
PTGENDER          1.0
AGE         76.666667
PTETHCAT          2.0
PTEDUCAT         18.0
PTRACCAT            5
Name: 1, dtype: object


In [147]:
#check age for nans 
# Check if there are any NaN values in the 'AGE' column
# Identify rows where the 'AGE' column has NaN values
rows_with_nans = adni_demo_filtered[adni_demo_filtered['AGE'].isna()]

# Display the rows with NaN values in 'AGE'
if not rows_with_nans.empty:
    print("Rows with NaN values in the 'AGE' column:")
    print(rows_with_nans)
else:
    print("No NaN values found in the 'AGE' column.")



Rows with NaN values in the 'AGE' column:
      RID  PTGENDER  AGE  PTETHCAT  PTEDUCAT PTRACCAT
98   4620       NaN  NaN       NaN       NaN      NaN
108  4767       NaN  NaN       NaN       NaN      NaN


Data dictionary codes for demo data adni dictionary 
* Gender codes: male = 1 , female = 2 
* Race codes: 1=American Indian or Alaskan Native; 2=Asian; 3=Native Hawaiian or Other Pacific Islander; 4=Black or African American; 5=White; 6=More than one race; 7=Unknown
* Ethnicity codes: 1=Hispanic or Latino; 2=Not Hispanic or Latino; 3=Unknown
* Education codes: not sure 


Manually add in the demographic data for 4620 and 4767 from the dx csv (this only has ethnicity not race)
* 4620 edu = 19 
* 4620 dx = nc 
* 4620 sex/gender = male (figure out the data dictionar code for male)
* 4620 age = 77 
* 4620 race = white (need to make sure the values match with the A4 values)
* 4620 eth = not hispanic 

* 4767 edu = 18
* 4767 dx = MCI 
* 4767 sex/gender = female
* 4767 age = 66 
* 4767 race = white 
* 4767 eth = not hispanic 


In [152]:
#add in missing data for 4767
correct_values_4767 = {
    'PTGENDER': 2,  #female 
    'AGE': 66,  
    'PTETHCAT': 2,  #not hs 
    'PTEDUCAT': 18,  
    'PTRACCAT': 5  # white
}

# Replace NaN values at index 98 with the correct values
for column, value in correct_values_4767.items():
    adni_demo_filtered.loc[98, column] = value

# Verify the changes
print(adni_demo_filtered.loc[98, ['PTGENDER', 'AGE', 'PTETHCAT', 'PTEDUCAT', 'PTRACCAT']])


PTGENDER     2.0
AGE         66.0
PTETHCAT     2.0
PTEDUCAT    18.0
PTRACCAT       5
Name: 98, dtype: object


In [153]:
#add in missing data for 4620 
correct_values_4620 = {
    'PTGENDER': 1,  #male
    'AGE': 77,  
    'PTETHCAT': 2,  #not hs 
    'PTEDUCAT': 19, 
    'PTRACCAT': 5  # white
}

# Replace NaN values at index 108 with the correct values
for column, value in correct_values_4620.items():
    adni_demo_filtered.loc[108, column] = value

# Verify the changes
print(adni_demo_filtered.loc[108, ['PTGENDER', 'AGE', 'PTETHCAT', 'PTEDUCAT', 'PTRACCAT']])

PTGENDER     1.0
AGE         77.0
PTETHCAT     2.0
PTEDUCAT    19.0
PTRACCAT       5
Name: 108, dtype: object


In [154]:
#check that there are no nans in the dataframe 
# Check if there are any NaN values in the entire DataFrame
has_nans = adni_demo_filtered.isna().any().any()

if has_nans:
    print("There are NaN values in the DataFrame.")
else:
    print("There are no NaN values in the DataFrame.")


There are no NaN values in the DataFrame.


In [148]:
print(adni_demo_filtered)

      RID  PTGENDER        AGE  PTETHCAT  PTEDUCAT PTRACCAT
0      31       2.0  83.750000       2.0      18.0        5
1     112       1.0  76.666667       2.0      18.0        5
2     120       1.0  79.083333       2.0      20.0        5
3     210       2.0  78.500000       2.0      13.0        5
4     377       1.0  85.166667       2.0      16.0        5
..    ...       ...        ...       ...       ...      ...
393  7054       2.0  72.250000       2.0      18.0        5
394  7079       2.0  86.750000       2.0      18.0        4
395  7088       1.0  70.000000       2.0      16.0        5
396  7100       1.0  74.000000       2.0      16.0        4
397  7105       1.0  77.500000       1.0      18.0        6

[398 rows x 6 columns]


IV. Now add in dx information:
* All of A4 data is CN (which is 1 in the adni data dictionary so add in a new row and just fill in rows with 1)
* For ADNI just load in dx_medhis.csv and take the diagnosis from there (1=NL;2=MCI;3=AD)

In [156]:
#all a4 is CN (1) 
a4_demo_filtered['DX']=1

In [157]:
print(a4_demo_filtered)

           RID  PTGENDER  PTAGE  PTETHNIC  PTEDUCAT PTRACE  DX
0    B10423472         2  78.25         2      19.0      5   1
1    B10693248         2  65.94         2      16.0      5   1
2    B10709603         1  74.15         2      18.0      5   1
3    B10795731         1  69.23         2      18.0      5   1
4    B10904689         2  68.21         2      13.0      5   1
..         ...       ...    ...       ...       ...    ...  ..
368  B99525373         2  67.00         2      20.0      5   1
369  B99649691         2  65.31         2      16.0      5   1
370  B99696480         2  74.41         2      12.0      5   1
371  B99860801         1  71.56         2      18.0      2   1
372  B99870985         1  71.32         2      14.0      5   1

[373 rows x 7 columns]


In [163]:
adni_dx = pd.read_csv('../../data_paths_and_cleaning/data/demographic_csvs/ADNI/adni_dx_and_medhis.csv')

In [164]:
print(adni_dx)

      NC  MCI  DE  AD  LBD  DLB  PDD   VD  FTD  faq_EVENTS  ...  faq_TRAVEL  \
0      0    1   0   0  0.0  0.0  0.0  0.0  0.0         2.0  ...         0.0   
1      1    0   0   0  0.0  0.0  0.0  0.0  0.0         0.0  ...         0.0   
2      0    1   0   0  0.0  0.0  0.0  0.0  0.0         2.0  ...         1.0   
3      0    1   0   0  0.0  0.0  0.0  0.0  0.0         0.0  ...         1.0   
4      0    1   0   0  0.0  0.0  0.0  0.0  0.0         0.0  ...         0.0   
...   ..  ...  ..  ..  ...  ...  ...  ...  ...         ...  ...         ...   
1816   0    1   0   0  0.0  0.0  0.0  0.0  0.0         0.0  ...         0.0   
1817   0    1   0   0  0.0  0.0  0.0  0.0  0.0         0.0  ...         1.0   
1818   0    1   0   0  0.0  0.0  0.0  0.0  0.0         0.0  ...         0.0   
1819   0    1   0   0  0.0  0.0  0.0  0.0  0.0         1.0  ...         0.0   
1820   0    1   0   0  0.0  0.0  0.0  0.0  0.0         0.0  ...         0.0   

      bat_DIGIF  his_HYPERTEN  bat_ANIMALS  faq_GAM

In [166]:
dx_cols = ['NC','MCI','DE','AD']
adni_dx=adni_dx[dx_cols]


In [169]:
#check for nans 
has_nans = adni_dx.isna().any().any()

if has_nans:
    print("There are NaN values in the DataFrame.")
else:
    print("There are no NaN values in the DataFrame.")

There are no NaN values in the DataFrame.


In [170]:
#replace the dx info from dx csv on demo filtered csv in single dx col 
def set_dx(row):
    if row['NC'] == 1:
        return 1
    elif row['MCI'] == 1:
        return 2
    elif row['AD'] == 1 or row['DE'] == 1:  # 3 for if they are ad or de = 1 
        return 3
    return None 

#fill in the values 
adni_demo_filtered['DX'] = adni_dx.apply(set_dx, axis=1)


In [174]:
# Calculate the minimum and maximum values in the 'DX' column
min_value = adni_demo_filtered['DX'].min()
max_value = adni_demo_filtered['DX'].max()

# Print the range of values in the 'DX' column
print(f"The range of values in the 'DX' column is from {min_value} to {max_value}.")

The range of values in the 'DX' column is from 1 to 3.


In [ ]:
#something like if ['NC']=1 then ['DX']=1, if 
for row in adni_dx:
    if adni_dx['NC']==1:
        adni_demo_filtered['DX']=1
    if adni_dx['MCI']==1:
        adni_demo_filtered['DX']=2
    if adni_dx['AD']==1:
        adni_demo_filtered['DX']=3
    if adni_dx['DE']==1:
        adni_demo_filtered['DX']=3

All good! The size of the demo dfs match so should be all good. Now save the demo csvs 

In [149]:
adni_demo_filtered.to_csv('../data/demographic_csvs/ADNI/adni_filtered_demo.csv')
a4_demo_filtered.to_csv('../data/demographic_csvs/A4/a4_filtered_demo.csv')